# NY Motor Vehicle Collisions – Exploratory Data Analysis  

This notebook focuses on the analysis of New York City's vehicular collision data, which is publicly available on the official New York City Open Data platform at https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95. The dataset contains information about motor vehicle crashes that occurred within the city.



Comprehending the dynamics of motor vehicle collisions is crucial for promoting public safety and providing essential information for policy decisions. The dataset offers a comprehensive overview of vehicular accidents, capturing detailed information such as date, time, location, contributing factors, and outcomes. This analysis aims to extract valuable insights from the data, serving policymakers, law enforcement, and the public. The goal is to foster a nuanced understanding of the factors influencing motor vehicle incidents, ultimately reducing accident occurrences and enhancing road safety.


The data contains information from all police-reported motor vehicle collisions in NYC. The information for this dataset is collated from the police report, called MV104-AN, which is required to be filled out for collisions where someone is injured or killed, or where there is at least $1000 worth of damage.


Data is available from 2012-07-01 onwards, however for this analysis, we will limit ourselves to the period up to 2023-08-15, which is when the data was downloaded.
We find that this data contains over 2 million observations, which allows us to explore several aspects of vehicle crashes in NY's boroughs.


The data dictionary for the data is also available at the URL above.


By leveraging exploratory data analysis (EDA) techniques and visualization tools, we aim to uncover insights into the factors contributing to collisions, identify high-risk areas, and provide valuable information for policymakers, law enforcement, and the general public. The analysis will be conducted using Python and popular data science libraries such as Pandas, Matplotlib, and Seaborn.


First, we perform unstructured exploration of the data, and then try to answer the following questions:

- We relate the number of crashes to shethe borough's population to find out which borough has the maximum number of crashes for every 100,000 people. Even though the data does not have this information, we can combine the crash data with the population estimates for the boroughs also available from the City of New York's website (https://data.cityofnewyork.us/City-Government/New-York-City-Population-by-Borough-1950-2040/xywu-7bv9)

| Borough | Population |
| --- | --- |
| Bronx | 1446788 |
| Brooklyn | 2648452 |
| Manhattan | 1638281 |
| Queens | 2330295 |
| Staten Island | 487155 | 

- Analyze the data to find the borough with the second-highest total number of crashes since 2012.
- Identify the leading cause of crashes, excluding cases where the contributing factor is 'Unspecified', and calculate the proportion of accidents attributable to this cause.
- Determine the top 5 causes of crashes (excluding 'Unspecified') and calculate the proportion of total crashes caused by these top 5 factors.
- Analyze the dataset to determine the total number of accidents reported since 2012 where the incident resulted in two or more fatalities.
- Conduct an analysis to determine the average fatality rate per 1000 accidents, showcasing how frequently at least one person succumbs to injuries in the dataset.
- Perform an analysis to calculate the proportion of accidents within the dataset that lack information on the Cross Street Name.
- Identify the combination of vehicles (represented by 'VEHICLE TYPE CODE 1' and 'VEHICLE TYPE CODE 2') with the most number of accidents.
- Among alcohol-related crashes, calculate the proportion that resulted in a fatality.
- Calculate the proportion of crashes that occur during the evening rush hour (4 PM to 7 PM).
- Among motorcycle crashes, calculate the proportion that resulted in injuries but no fatalities.
- Analyze the dataset to calculate the total number of crashes involving bicycles as one of the vehicles, providing insights into the frequency of such incidents.
- Combine crash data with population estimates and determine the borough with the most crashes for every 100,000 people.


In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
import phik
from IPython.display import Markdown as md

In [3]:
df = pd.read_pickle(r"Motor_Vehicle_Collisions_-_Crashes.pkl")
df

,CRASH DATE_CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2021-09-11 02:39:00,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,2.0,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,2022-03-26 11:45:00,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,2022-06-29 06:55:00,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,2021-09-11 09:35:00,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,0.0,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,2021-12-14 08:13:00,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018240,2023-07-03 18:05:00,NaN,NaN,40.866806,-73.931010,"(40.866806, -73.93101)",RIVERSIDE DRIVE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4648110,Sedan,Sedan,NaN,NaN,NaN
2018241,2023-07-22 21:39:00,BRONX,10457.0,40.844177,-73.902920,"(40.844177, -73.90292)",EAST 174 STREET,WEBSTER AVENUE,NaN,1.0,...,NaN,NaN,NaN,NaN,4648117,Sedan,NaN,NaN,NaN,NaN
2018242,2023-07-02 17:55:00,MANHATTAN,10006.0,40.711033,-74.014540,"(40.711033, -74.01454)",WEST STREET,LIBERTY STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4648366,Taxi,NaN,NaN,NaN,NaN
2018243,2023-07-22 13:15:00,QUEENS,11433.0,40.691580,-73.793190,"(40.69158, -73.79319)",110 AVENUE,157 STREET,NaN,1.0,...,Driver Inattention/Distraction,NaN,NaN,NaN,4648129,Station Wagon/Sport Utility Vehicle,E-Bike,NaN,NaN,NaN


## Borough with the second highest number of crashes reported since 2012

In [9]:
df['CRASH DATE_CRASH TIME'] = pd.to_datetime(df['CRASH DATE_CRASH TIME'])
df_since_2012 = df[df['CRASH DATE_CRASH TIME'].dt.year >= 2012]

In [10]:
# Group by borough and calculate the total number of crashes reported for each borough
borough_crashes_total = df_since_2012.groupby('BOROUGH')['NUMBER OF PERSONS INJURED'].sum()


In [11]:
# Find the borough with the second-highest total number of crashes reported
second_highest_borough = borough_crashes_total.sort_values(ascending=False).index[1]


In [12]:
print(f"The borough with the second-highest total number of crashes reported since 2012 is: {second_highest_borough}")

The borough with the second-highest total number of crashes reported since 2012 is: QUEENS


## Borough has the most crashes for every 100,000 people since July 2012

In [13]:
# Define the population for each borough
population = pd.Series({
    'BRONX': 1446788,
    'BROOKLYN': 2648452,
    'MANHATTAN': 1638281,
    'QUEENS': 2330295,
    'STATEN ISLAND': 487155
})


In [14]:
# Calculate the total number of crashes in each borough
borough_crashes = df['BOROUGH'].value_counts()


In [15]:
# Calculate the number of crashes per 100,000 people for each borough
crashes_per_capita = (borough_crashes / population) * 100000


In [16]:
# Get the borough with the most crashes per 100,000 people
borough_most_crashes_per_capita = crashes_per_capita.idxmax()
most_crashes = crashes_per_capita.max()


In [17]:
print(f"{borough_most_crashes_per_capita}, with {most_crashes:.5f} crashes since July 2012 per 100k population")

MANHATTAN, with 19121.62810 crashes since July 2012 per 100k population


## Analyzing the leading cause of crashes

In [19]:
# Filter out rows where 'CONTRIBUTING FACTOR VEHICLE 1' is 'Unspecified'
df_filtered = df[df['CONTRIBUTING FACTOR VEHICLE 1'] != 'Unspecified']

# Determine the leading cause of crashes
leading_cause = df_filtered['CONTRIBUTING FACTOR VEHICLE 1'].mode().values[0]

# Calculate the proportion of accidents attributable to the leading cause
proportion = (df_filtered['CONTRIBUTING FACTOR VEHICLE 1'].value_counts().loc[leading_cause] / len(df_filtered)) * 100

print(f"The leading cause of crashes (ignoring 'Unspecified') is: {leading_cause}")

print(f"The proportion of accidents attributable to this cause is: {proportion:.2f}%")


The leading cause of crashes (ignoring 'Unspecified') is: Driver Inattention/Distraction
The proportion of accidents attributable to this cause is: 30.27%


## Top 5 causes of crashes

In [21]:
# Filter out rows where 'CONTRIBUTING FACTOR VEHICLE 1' is 'Unspecified'
df_filtered = df[df['CONTRIBUTING FACTOR VEHICLE 1'] != 'Unspecified']

# Get the top 5 causes of crashes
top_5_causes = df_filtered['CONTRIBUTING FACTOR VEHICLE 1'].value_counts().head(5).index

# Filter the dataframe for the top 5 causes
df_top_5_causes = df_filtered[df_filtered['CONTRIBUTING FACTOR VEHICLE 1'].isin(top_5_causes)]

# Calculate the proportion of total crashes for the top 5 causes
proportion_top_5_causes = (len(df_top_5_causes) / len(df_filtered)) * 100

print(f"The top 5 causes of crashes (ignoring 'Unspecified') account for {proportion_top_5_causes:.2f}% of total crashes.")


The top 5 causes of crashes (ignoring 'Unspecified') account for 57.76% of total crashes.


## Total count of accidents that involved two or more fatalities

In [23]:
# Assume missing values for the number of persons killed are equal to zero killed
df['NUMBER OF PERSONS KILLED'] = df['NUMBER OF PERSONS KILLED'].fillna(0)

# Filter for accidents with two or more fatalities
accidents_two_or_more_fatalities = df[df['NUMBER OF PERSONS KILLED'] >= 2]

# Calculate the total count of accidents with two or more fatalities
total_accidents_two_or_more_fatalities = len(accidents_two_or_more_fatalities)

print(f"The total count of accidents since 2012 that involved two or more fatalities is: {total_accidents_two_or_more_fatalities}")


The total count of accidents since 2012 that involved two or more fatalities is: 88


##  The average number of accidents resulting in at least one person dead for every 1000 accidents

In [25]:
# Assume missing values for the number of persons killed are equal to zero killed
df['NUMBER OF PERSONS KILLED'] = df['NUMBER OF PERSONS KILLED'].fillna(0)

# Calculate the count of accidents with 1 or more deaths
accidents_with_one_or_more_deaths = df[df['NUMBER OF PERSONS KILLED'] >= 1]

# Calculate the total number of accidents
total_accidents = len(df)

# Calculate the average number of accidents with at least one person dead for every 1000 accidents
average_accidents_with_deaths_per_1000 = (len(accidents_with_one_or_more_deaths) / total_accidents) * 1000

print(f"On average, for every 1000 accidents, approximately {average_accidents_with_deaths_per_1000:.3f} have resulted in at least one person dead.")


On average, for every 1000 accidents, approximately 1.389 have resulted in at least one person dead.


## The proportion of accidents in the data that do not have a Cross Street Name

In [27]:
# Calculate the proportion of accidents without a Cross Street Name
proportion_without_cross_street = (df['CROSS STREET NAME'].isnull().sum() / len(df)) * 100

print(f"The proportion of accidents without a Cross Street Name is: {proportion_without_cross_street:.1f}%")


The proportion of accidents without a Cross Street Name is: 37.4%


##  Vehicle Combinations in Motor Vehicle Accidents

In [29]:
# Drop rows where both 'VEHICLE TYPE CODE 1' and 'VEHICLE TYPE CODE 2' are missing
df_vehicle_combinations = df.dropna(subset=['VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2']).copy()

# Concatenate 'VEHICLE TYPE CODE 1' and 'VEHICLE TYPE CODE 2' to represent the combination of vehicles
df_vehicle_combinations['VEHICLE COMBINATION'] = df_vehicle_combinations['VEHICLE TYPE CODE 1'] + ' / ' + df_vehicle_combinations['VEHICLE TYPE CODE 2']

# Find the combination of vehicles with the most number of accidents
most_common_vehicle_combination = df_vehicle_combinations['VEHICLE COMBINATION'].mode().values[0]

# Count the number of accidents for the most common vehicle combination
most_common_vehicle_combination_count = df_vehicle_combinations[df_vehicle_combinations['VEHICLE COMBINATION'] == most_common_vehicle_combination].shape[0]

print(f"The combination of vehicles '{most_common_vehicle_combination}' has the most number of accidents with a count of {most_common_vehicle_combination_count}.")


The combination of vehicles 'Sedan / Sedan' has the most number of accidents with a count of 197944.


## Proportion of Crashes with Alcohol Involvement Resulting in Fatality

In [31]:
# Filter for collisions where the contributing factor was Alcohol Involvement
alcohol_related_collisions = df[df['CONTRIBUTING FACTOR VEHICLE 1'] == 'Alcohol Involvement']

# Calculate the proportion of alcohol-related collisions that resulted in a fatality
proportion_fatalities_alcohol_related = (alcohol_related_collisions['NUMBER OF PERSONS KILLED'].sum() / len(alcohol_related_collisions)) * 100

print(f"The proportion of crashes where the contributing factor was Alcohol Involvement that resulted in a fatality is: {proportion_fatalities_alcohol_related:.1f}%")


The proportion of crashes where the contributing factor was Alcohol Involvement that resulted in a fatality is: 0.5%


## The proportion of crashes during the evening rush hour (4 PM to 7 PM)

In [33]:
# Filter for crashes that occur between 4 PM and 7 PM
evening_rush_hour_collisions = df[(df['CRASH DATE_CRASH TIME'].dt.hour >= 16) & (df['CRASH DATE_CRASH TIME'].dt.hour < 19)]

# Calculate the proportion of crashes during the evening rush hour
proportion_evening_rush_hour = (len(evening_rush_hour_collisions) / len(df)) * 100

print(f"The proportion of crashes during the evening rush hour (4 PM to 7 PM) is: {proportion_evening_rush_hour:.1f}%")


The proportion of crashes during the evening rush hour (4 PM to 7 PM) is: 20.5%


## The proportion of crashes involving motorcycles that resulted in injuries but no fatalities

In [36]:
# Filter for crashes involving motorcycles
motorcycle_collisions = df[df['VEHICLE TYPE CODE 1'].str.contains('MOTORCYCLE') | df['VEHICLE TYPE CODE 2'].str.contains('MOTORCYCLE')]

# Filter the reduced dataset for injuries greater than 0 and fatalities equal to 0
injuries_no_fatalities_motorcycle = motorcycle_collisions[(motorcycle_collisions['NUMBER OF PERSONS INJURED'] > 0) & (motorcycle_collisions['NUMBER OF PERSONS KILLED'] == 0)]

# Calculate the proportion of motorcycle crashes resulting in injuries but no fatalities
proportion_injuries_no_fatalities_motorcycle = (len(injuries_no_fatalities_motorcycle) / len(motorcycle_collisions)) * 100

print(f"The proportion of crashes involving motorcycles that resulted in injuries but no fatalities is: {proportion_injuries_no_fatalities_motorcycle:.2f}%")


The proportion of crashes involving motorcycles that resulted in injuries but no fatalities is: 50.05%


## The number of crashes involving bicycles as one of the vehicles


In [38]:
# Filter for crashes involving bicycles
bicycle_collisions = df[df['VEHICLE TYPE CODE 1'].str.contains('BICYCLE') | df['VEHICLE TYPE CODE 2'].str.contains('BICYCLE')]

# Calculate the number of crashes involving bicycles
num_bicycle_collisions = len(bicycle_collisions)

print(f"The number of crashes involving bicycles as one of the vehicles is: {num_bicycle_collisions}")

The number of crashes involving bicycles as one of the vehicles is: 19108
